In [1]:
# check copyright of the game (you bought the game, right?)
urbek_data = r"H:\Steam\steamapps\common\Urbek\Urbek_Data"
with open(fr"{urbek_data}\app.info") as f:
    print(f.read())

Estudios Kremlinois
Urbek


In [2]:
%%html
<style>
  #notebook-container {
      width: 99%;
  }
  body,
  div#notebook,
  div.output_subarea {
    overflow: visible; /* for the `position: sticky` */
  }
  .dataframe {
      position: relative;
  }
  .dataframe thead {
      position: sticky;
      top: 0;
      background: black;
      z-index: 1;
  }
  .dataframe thead tr:nth-child(2) {
      height: 250px;
  }
  .dataframe thead th {
      white-space: pre;
      vertical-align: bottom;
      text-align: left !important;
  }
  .dataframe thead  tr:nth-child(2) th:nth-child(n+5) {
      transform-origin: bottom left;
      transform: translateX(20px) rotate(-60deg);
      max-width: 20px;
  }
  .dataframe td {
      white-space: pre;
      border-left: 1px dotted #333;
  }
  .dataframe td:nth-child(4) {
      position: sticky;
      left: 0;
      background: inherit;
  }
  .dataframe td:nth-child(n+5) {
      text-align: right;
  }
</style>

In [3]:
import os
import re
import json
import pandas
import UnityPy
import numpy as np

pandas.set_option('display.max_rows', 1000)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_colwidth', None)

In [4]:
# load data file from game installation
env = UnityPy.load(fr"{urbek_data}\resources.assets")

rules = {}
translations = {}
for obj in env.objects:
    if obj.type.name != "TextAsset": continue
    data = obj.read()
#     # download json files for browsing:
#     os.makedirs("./data", exist_ok=True)
#     with open(f"./data/{data.name}.json", "wb") as f:
#         f.write(data.script)
    if data.name == "gameValues":
        for item in json.loads(data.text):
            translations[item["id"]] = item
    if re.match(r"reglasBioma_\d$", data.name) and \
       re.search(r"localVariables", data.text):
        rules[data.name] = json.loads(data.text)
del env, obj, data

# explore list of languages
translations["resources_comida"]

{'id': 'resources_comida',
 'INFO': 'name of the resources',
 'en': 'Food',
 'es': 'Comida',
 'fr': 'Nourriture',
 'zh-CN': '食物',
 'pl': 'Jedzenie ',
 'de': 'Nahrungsmittel  ',
 'pt': 'Comida',
 'ru': 'Еда',
 'ja': '食品',
 'it': 'Alimentazione',
 'ko': '음식'}

In [5]:
# choose a language
language = "en"
labels = dict((k, v[language])
              for k, v in translations.items()
              if not re.search(r"text|desc|name|effect|t_|nr_", k))

# explore list of (non-empty) biomes
dict(sorted([(k, re.sub(r"<b>|</b>[\s\S]*", "", v["description"])) for k, v in rules.items()]))

{'reglasBioma_0': 'Temperate',
 'reglasBioma_1': 'Desert',
 'reglasBioma_2': 'Archipelago',
 'reglasBioma_3': 'Forest',
 'reglasBioma_4': 'Ruins'}

In [6]:
# choose a biome
biome = "reglasBioma_0"
data = rules[biome]
columns_resources = [item.split(',')[0] for item in data["resources"]]
columns_lv = [item.split(',')[0] for item in data["localVariables"]]

# parse buildings form the biome
buildings = []
for building in data["construs"]:
    codeName = building['codeName']
    label = labels.get(f'b_{codeName}', '???')
    category = building.get('category')
    category = f"{category} ({labels.get(f'bg_{category}')})" if category else ""
    distanceToRoad = building.get('distanceToRoad', '')
    produces = [next((p['q']
                      for p in building.get('produces', [])
                      if p['rec'] == col), "")
                for col in columns_resources]
    lvn = [next((f"{'<= ' if not p['biggerThan'] else ''}{str(p['q']) + '□ ' + str(p['threshold']) if 'threshold' in p else p['q']} ⊙{p['radio']}"
                 for p in building.get('localVariablesNeeded', [])
                 if p['localVariable'] == col), "")
           for col in columns_lv]
    lv = [next((f"{p['q']}"
                 for p in building.get('localVariables', [])
                 if p['locVar'] == col), "")
          for col in columns_lv]
    
    if any([*lvn, *lv, *produces]):
        buildings.append([category,
                          codeName,
                          label,
                          distanceToRoad,
                          *lvn,
                          *produces,
                          *lv,
                         ])

# visualize buildings
columns = pandas.MultiIndex.from_tuples([*[("", "category"), ("", "codeName"), ("", f"Building name ({language})"), ("", "distanceToRoad")],
                                         *[("localVariablesNeeded", f"{labels[f'lv_{c}']}  ({c})") for c in columns_lv],
                                         *[("produces", f"{labels[f'resources_{c}']}  ({c})") for c in columns_resources],
                                         *[("localVariables", f"{labels[f'lv_{c}']}  ({c})") for c in columns_lv],
                                        ])
df = pandas.DataFrame(buildings, columns=columns)
df
# use SHIFT + scroll for horizontal scroll

\
                            category                   codeName   
0                     camino (Roads)               caminoPueblo   
1                     camino (Roads)           caminoIndustrial   
2                     camino (Roads)                caminoLindo   
3                     camino (Roads)          caminoPuebloLindo   
4                     camino (Roads)                caminoPobla   
5                     camino (Roads)            caminoComercial   
6                     camino (Roads)                    camino2   
7                     camino (Roads)                    camino3   
8                     camino (Roads)           caminoRuralLindo   
9                     camino (Roads)             caminoPeatonal   
10                    camino (Roads)            caminoPeatonal2   
11                                                        campo   
12                                                      cabanha   
13                                                  puebloPobre   
14                                                       pueblo   
15                                                        villa   
16                                                      fosforo   
17                    casas (Houses)                 pescadores   
18                    casas (Houses)                       HLM0   
19                    casas (Houses)                       HLM1   
20                    casas (Houses)                       HLM2   
21                                                   poblaPobre   
22                                                        pobla   
23                                                       ciudad   
24                                                      ciudad2   
25                                                      moderna   
26                                                        mixta   
27                                                       mixta5   
28                                                       mixta8   
29                                               edificio5verde   
30                                                  mixta5verde   
31                                                         lota   
32                                                        lota3   
33                                                   lotaCarbon   
34                                                  lotaCarbon3   
35                                                     callampa   
36                                                      rebelde   
37                                              estacionamiento   
38                                                      bloques   
39                   parques (Parks)                     parque   
40                   parques (Parks)                 parquePlus   
41                   parques (Parks)            plazaCondominio   
42                     campo (Farms)                casaDeCampo   
43                     campo (Farms)                 inquilinos   
44                     campo (Farms)                   patronal   
45                     campo (Farms)                       silo   
46                     campo (Farms)                siloModerno   
47                     campo (Farms)              protoFrutales   
48                     campo (Farms)                   frutales   
49                     campo (Farms)                  cobertizo   
50                     campo (Farms)                plantaFruta   
51                     campo (Farms)               plantaComida   
52             industrias (Industry)         campamentoMaderero   
53             industrias (Industry)                   lenhador   
54             industrias (Industry)               lenhadorPlus   
55             industrias (Industry)                 molinoAgua   
56             industrias (Industry)                     muelle   
57             industrias (Industry)                 minaHierro   
58             industrias (Industry)         